---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [34]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.iloc[321]

'2June, 1999 Audit C Score Current:\n'

In [41]:
from datetime import datetime
from dateutil.parser import parse
import numpy as np
# format: month/-day/-year

# V 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# V Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 
# V 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# V Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# V 6/2008; 12/2009
# V 2009; 2010
def date_sorter():
        # format: month/-day/-year

        # V 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        # V Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 
        # V 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        # V Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        # Feb 2009; Sep 2009; Oct 2010
        # V 6/2008; 12/2009
        # V 2009; 2010

        # Getting rid of some of the whitespace 
    # Getting rid of some of the whitespace 
    df_df=df.to_frame(name='text')
    df_df['text'] = df_df['text'].str.replace(r'[\n\t\f\v\r.,;)():]', '')
    df_df['index'] = df_df.index
    # df_df.iloc[72]

    #04/20/2009; 
    df1=df_df['text'].str.extractall(r'(?P<date>(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4}))').reset_index(level=[1,0, 0])[['level_0', 'year', 'month', 'day']]
    # 04/20/09; 4/20/09; 4/3/09
    # df1_b=df_df['text'].str.extractall(r'(?P<date>(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}))').reset_index(level=[1,0, 0])[['level_0', 'year', 'month', 'day']]
    # translate two digit year to 4 year
    # Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
    df1['year']= np.where(df1['year'].astype(int)<100, df1['year'].astype(int)+1900, df1['year'])


    # 6/2008; 12/2009
    df2=df_df['text'].str.extractall(r'(?P<date>(?P<month>\d{1,2})[/](?P<year>\d{4}))').reset_index(level=[1,0, 0])[['level_0', 'year', 'month']]
    df2=df2.loc[(df2['year'].astype(int)<=2020) & (df2['year'].astype(int)>=1900) & (df2['month'].astype(int)<=12)]
    df2['day']=1
    # year
    # df2=df['text'].str.extractall(r'(?P<year>([^;-]\D[a-z]*(\d{4})\D))')

    #  2009; 2010

    df3=df_df['text'].str.extractall(r'(?P<year>((([-/A-Za-z]?)\d{4})))').reset_index(level=[1,0, 0])[['level_0', 'year']]
    df3.set_index('level_0', inplace=True)
    df3=df3['year'].str.extractall(r'(?P<year>((\d{4})))').reset_index(level=[1,0])[['level_0', 'year']]
    df3=df3.loc[(df3['year'].astype(int)<=2020) & (df3['year'].astype(int)>=1900)]
    df3['month']=1
    df3['day']=1
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; V
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 V
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 V
    # Feb 2009; Sep 2009; Oct 2010V 
    d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12,}

    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    df4=df_df['text'].str.extractall(r'(?P<date_notcl>(\d{2}[ ]?(?: Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]*\d{4}))').reset_index(level=[1,0, 0])
    df4['date']=df4.date_notcl.str.replace(r"([0-9]+(\.[0-9]+)?)",r" \1 ").str.lstrip()
    df4['day'] = df4.date.str.lstrip().apply(lambda x:x.split(' ')[0])
    df4['month'] =df4.date.str.lstrip().apply(lambda x:x.split(' ')[2])
    df4['year'] = df4.date.str.lstrip().apply(lambda x:x.split(' ')[4])
    df4.month = df4['month'].str[0:3].map(d)
    df4=df4[['level_0', 'year', 'month', 'day']]

    df4_b=df_df['text'].str.extractall(r'(?P<date_notcl>(^\d{1}[ ]?(?: Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]*\d{4}))').reset_index(level=[1,0, 0])
    df4_b['date']=df4_b.date_notcl.str.replace(r"([0-9]+(\.[0-9]+)?)",r" \1 ").str.lstrip()
    df4_b['day'] = df4_b.date.str.lstrip().apply(lambda x:x.split(' ')[0])
    df4_b['month'] =df4_b.date.str.lstrip().apply(lambda x:x.split(' ')[1])
    df4_b['year'] = df4_b.date.str.lstrip().apply(lambda x:x.split(' ')[3])
    df4_b.month = df4_b['month'].str[0:3].map(d)
    df4_b=df4_b[['level_0', 'year', 'month', 'day']]

    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 
    df5=df_df['text'].str.extractall(r'(?P<date>((?: Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*[ ]*[-]?\d{1,2}[A-Za-z]*[-,]?[ ]*\d{4}))').reset_index(level=[1,0, 0])
    df5['month'] =df5.date.str.lstrip().apply(lambda x:x.split(' ')[0])
    df5['day'] = df5.date.str.lstrip().apply(lambda x:x.split(' ')[1]).str[0:2]
    df5['year'] = df5.date.str.lstrip().apply(lambda x:x.split(' ')[2])
    df5.month = df5['month'].str[0:3].map(d)
    df5=df5[['level_0', 'year', 'month', 'day']]


    # Feb 2009; Sep 2009; Oct 2010
    df6=df_df['text'].str.extractall(r'(?P<date>((?: Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*[ ]?\d{4}))').reset_index(level=[1,0, 0])
    df6['month'] =df6.date.str.lstrip().apply(lambda x:x.split(' ')[0])
    df6['day'] = 1
    df6['year'] = df6.date.str.lstrip().apply(lambda x:x.split(' ')[1])
    df6.month = df6['month'].str[0:3].map(d)
    df6=df6[['level_0', 'year', 'month', 'day']]


    best3= pd.concat([df1,  df4, df4_b, df5])
    best3['best_data']=3
    df2['best_data']=2
    df6['best_data']=2
    df3['best_data']=1
    data = pd.concat([best3, df2, df3, df6])


    data=data.loc[(data['month'].astype(int)<=12) & (data['day'].astype(int)<=31)]
    data['date']= data['month'].astype(int).astype(str)+'/'+ data['day'].astype(int).astype(str)+'/'+data['year'].astype(int).astype(str)
    data['date_num']=pd.to_datetime(data['date'], format='%m/%d/%Y')
    # data=pd.DataFrame(data.groupby(['level_0'], sort=False)['best_data'].max())
    idx = data.groupby(['level_0'])['best_data'].transform(max) == data['best_data']
    data=data[idx].sort_values(by=['level_0'], ascending=True)
    # df1['date']=pd.to_datetime(dict(year=df1.year, month=df1.month, day=df1.day), format='%y-%m-%d')
    # data['date']= data['month'].astype(int).astype(str)+'/'+ data['day'].astype(int).astype(str)+'/'+data['year'].astype(int).astype(str)
    # data2=data.loc[data['month'].astype(int)<=12]
    # data['date_num']=pd.to_datetime(data['date'], format='%m/%d/%Y')
    # data['date']=date(data['year'].astype(int), data['month'].astype(int), data['day'].astype(int))
    data_min=pd.DataFrame(data.groupby(['level_0'], sort=False)['date_num'].min()).sort_index()
    ranked = pd.Series([i[0] for i in sorted(enumerate(data_min['date_num']), key=lambda x:x[1])],np.arange(500))
    ranked

    return ranked
print(date_sorter())

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64
